In [1]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import re
import time

In [64]:


headers = {
    "USER_AGENT": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36",
    "referer": "https://fincaraiz.com.co/"
}

json_data = {
    'filter': {
        'offer': {
            'slug': [
                'sell',
            ],
        },
        'property_type': {
            'slug': [
                'apartment',
            ],
        },
        'locations': {
            'location_point':[
                [
                -76.59027692154969,
                3.535311581094419
                ],
                [
                -76.48934003190125,
                3.3689308955117525
                ],
            ],
        },
    },
    'fields': {
        'exclude': [],
        'include': [
            'area',
            'baths.id',
            'baths.name',
            'client.client_type',
            'client.company_name',
            'client.first_name',
            'client.last_name',
            'garages.name',
            'is_new',
            'locations.cities.name',
            'locations.cities.slug',
            'locations.countries.name',
            'locations.countries.slug',
            'locations.groups.name',
            'locations.groups.slug',
            'locations.groups.subgroups.name',
            'locations.groups.subgroups.slug',
            'locations.location_point',
            'locations.neighbourhoods.name',
            'locations.neighbourhoods.slug',
            'locations.states.name',
            'locations.states.slug',
            'locations.view_map.slug',
            'media.floor_plans.count',
            'media.photos.list.id',
            'media.photos.list.image.full_size',
            'media.photos.list.is_main',
            'media.videos.count',
            'min_area',
            'min_price',
            'price',
            'products.configuration.tag_name',
            'products.label',
            'products.name',
            'products.slug',
            'property_id',
            'fr_property_id',
            'rooms.name',
            'title',
            'property_type.name',
            'offer.name',
            'fr_parent_property_id',
        ],
        'limit': 25,
        'offset': 0,
        'ordering': [],
        'platform': 40,
        'with_algorithm': False,
    }
}

response = requests.post('https://api.fincaraiz.com.co/document/api/1.0/listing/search', headers=headers, json=json_data, timeout=10)
data = response.json()

### Pruebas para implementar paginación

In [61]:
def fetch_data(offset):
    # Establece el offset para obtener el conjunto de resultados correspondiente
    json_data['fields']['offset'] = offset

    # Realiza la solicitud a la API
    response = requests.post('https://api.fincaraiz.com.co/document/api/1.0/listing/search', headers=headers, json=json_data, timeout=10)
    data = response.json()

    return data

In [ ]:
# Define el límite de resultados por página (máximo 100 según la API)
limit_per_page = 100

# Lista para almacenar los datos de todas las páginas
all_properties_data = []

# Número total de resultados que deseas obtener
total_results = 1000  # Por ejemplo, para obtener 1000 resultados

# Itera para obtener cada página de resultados
for offset in range(0, total_results, limit_per_page):
    data = fetch_data(offset)
    properties_data = data['hits']['hits']

    # Extrae la información relevante y agrega a la lista
    for hit in properties_data:
        source = hit['_source']['listing']
        # ... (código para extraer información y crear property_info) ...

    all_properties_data.extend(properties_data)

# Crear DataFrame con todos los datos recopilados
df = pd.DataFrame(all_properties_data)

### Fin de la prueba

In [60]:
data

{'took': 17,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7181, 'relation': 'eq'},
  'max_score': 0.0,
  'hits': [{'_index': 'fr-site-listing',
    '_id': '144f2bdc-cc13-4aa8-9fd1-bf5737e81237',
    '_score': 0.0,
    '_source': {'listing': {'area': '57.66',
      'rooms': {'name': '2'},
      'is_new': False,
      'media': {'videos': [{'count': 0}],
       'photos': [{'list': [{'image': {'full_size': 'https://s3.amazonaws.com/imagenesprof.fincaraiz.com.co/OVFR_COL/2022/3/29/3613035_585_0.jpg'},
           'is_main': True,
           'id': 'eea12944-c652-47dd-b2cc-dc3b61c24a50'},
          {'image': {'full_size': 'https://s3.amazonaws.com/imagenesprof.fincaraiz.com.co/OVFR_COL/2022/3/29/3613035_646_1.jpg'},
           'is_main': False,
           'id': '3a58ef0b-8f15-4e3d-be64-c48a55ac95a8'},
          {'image': {'full_size': 'https://s3.amazonaws.com/imagenesprof.fincaraiz.com.co/OVFR_COL/2022/3/29/3613035_102

In [65]:
properties_data = []
for hit in data['hits']['hits']:
    source = hit['_source']['listing']
    property_info = {
        'area': source['area'],
        'rooms': source['rooms']['name'],
        'is_new': source['is_new'],
        'property_id': source['property_id'],
        'garages': source['garages']['name'],
        'baths': source['baths']['name'],
        'min_price': source['min_price'],
        'price': source['price'],
        'company_name': source['client']['company_name'],
        'client_type': source['client']['client_type'],
        'neighbourhood': source['locations']['neighbourhoods'][0]['name'],
        'city': source['locations']['cities'][0]['name'],
        'latitude': source['locations']['location_point'].split()[1][1:],
        'longitude': source['locations']['location_point'].split('(')[1].split()[1].rstrip(')'),
        'zone': source['locations'].get('groups', [{'subgroups': {'name': 'No Zone'}}])[0]['subgroups']['name']
    }
    properties_data.append(property_info)

# Crear DataFrame
df = pd.DataFrame(properties_data)

df

,area,rooms,is_new,property_id,garages,baths,min_price,price,company_name,client_type,neighbourhood,city,latitude,longitude,zone
0,151.0,4,False,917b3c22-0881-4118-a3f5-4f9579ee433e,2,5,0,750000000.0,Logros Inmobiliarios,BROKER,Bellavista,Cali,-76.55243569999999,3.449486,No Zone
1,88.0,3,False,61573c0f-ddec-443c-a55a-13c91013fe9a,1,2,0,360000000.0,Sansilvestre Group Inmobiliaria,BROKER,La Hacienda,Cali,-76.53096008300781,3.3957936763763428,Zona Sur
2,103.0,3,False,c7a7bafa-e8c3-4f94-93ca-90a776039598,2,3,0,315000000.0,Sansilvestre Group Inmobiliaria,BROKER,Valle del Lili,Cali,-76.52033996582031,3.378350257873535,Zona Sur
3,189.0,3,False,ae2fcbbb-8fc7-4269-a2de-4b37136b425c,2,3,0,535000000.0,Sansilvestre Group Inmobiliaria,BROKER,Quintas de Don Simón,Cali,-76.54238891601562,3.395352840423584,Zona Sur
4,72.0,2,False,8e038ac1-1c96-475f-adea-20b1b0b7cc99,1,1,0,150000000.0,Sansilvestre Group Inmobiliaria,BROKER,Santa Anita,Cali,-76.53836059570312,3.405600070953369,Zona Sur
5,85.0,3,False,27aec250-f533-4335-8724-8f0df99413a8,1,2,0,220000000.0,Sansilvestre Group Inmobiliaria,BROKER,El Refugio,Cali,-76.55159759521484,3.3981409072875977,Zona Sur
6,75.0,3,False,8b8dbd37-8db0-449b-8f48-4b173a3e66ea,1,2,0,310000000.0,Somos Brokers Inmobiliarios,BROKER,El lido,Cali,-76.54900360107422,3.4240000247955322,No Zone
7,142.0,4,False,8b829902-94a9-4919-8712-dc6db1b4ac85,2,4,0,530000000.0,Bolsa De Propiedades,BROKER,Bellavista,Cali,-76.54867553710938,3.446683645248413,Zona Oeste
8,77.0,3,False,7a0a1039-9145-4fbe-925d-352172664be1,1,2,0,285000000.0,Bolsa De Propiedades,BROKER,Sur Valle Del Lili,Cali,-76.51705169677734,3.369626760482788,No Zone
9,53.0,2,False,17710103-0e46-40c7-808a-66b6cd90c20d,Sin especificar,2,0,170000000.0,Artemo y Bienes S.A.S,BROKER,Valle del Lili,Cali,-76.53777313232422,3.446890115737915,No Zone
